In [6]:
import numpy as np
import os
import glob
import topofisher
from topofisher.filtrations.numpy.alphaDTML import AlphaDTMLayer
from topofisher.filtrations.numpy.default_filtrations import AlphaFiltration
from topofisher.vectorizations.numpy.custom_vectorizations import TOPK
from topofisher.vectorizations.numpy.vectorization_layer import VectorizationLayers
import matplotlib.pyplot as plt
from topofisher.fisher.Fisher import show_fm_and_bias, fisherMOPED, fisherFromVecs
import tensorflow as tf
import importlib
importlib.reload(topofisher.fisher.imnn)
from topofisher.fisher.imnn import IMNNLayer, MopedLayer, ExtraDimLayer
import pickle
import glob
import tqdm
from tqdm import tqdm

# Functions, benchmarks and familiarizing with data

## Some functions

In [7]:
def shuffle_along_axis(arr, seed = None):
    if seed is None: seed = np.random.randint(1e4)
    # Create a random number generator with the specified seed
    rng = np.random.RandomState(seed)

    # Use the random number generator to shuffle the array along the first axis
    shuffled_arr = arr[rng.permutation(arr.shape[0])]
    return shuffled_arr

def get_vecs(directory_path):
    # Find all pickle files in the directory
    pickle_files = glob.glob(os.path.join(directory_path, '*.pkl'))
    fid_vecs = []
    der_vecs = {}
    param_list = ["Om_m", "Om_p", "s8_m", "s8_p"]
    for item in param_list : der_vecs[item] = []
    for file_path in pickle_files:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
            fid_vecs.append(data[0]['fiducial'])
            # print(data[0].keys())
            for item in param_list : der_vecs[item].append(data[1][item])

    fid_arr = np.array(np.concatenate(fid_vecs, axis = 0), dtype = np.float64)
    # der_arr = np.array([ for item in param_list], dtype = np.float64)
    der_arr = {}
    for item in param_list: der_arr[item] = np.array(np.concatenate(der_vecs[item], axis = 0), dtype = np.float64)
    return [fid_arr, der_arr['Om_m'], der_arr['Om_p'], der_arr['s8_m'], der_arr['s8_p']]


def fisherAnalysisMOPED(directory_path):
    # Find all pickle files in the directory
    arrs = get_vecs(directory_path)
    seeds = [np.random.randint(100) for _ in range(3)]
    fid_arr = shuffle_along_axis(arrs[0], seed = seeds[0])
    der_arr = [shuffle_along_axis(arrs[idx + 1], seeds[idx//2 + 1]) for idx in range(4)]
    fish = fisherMOPED([fid_arr, *der_arr], np.array([0.02, 0.03]))
    show_fm_and_bias(fish)
    return fish

def fisherAnalysis(directory_path):
    # Find all pickle files in the directory
    arrs = get_vecs(directory_path)
    seeds = [np.random.randint(100) for _ in range(3)]
    # fid_arr, der_arr = arrs[0], arrs[1:]
    fid_arr = shuffle_along_axis(arrs[0], seed = seeds[0])
    der_arr = [shuffle_along_axis(arrs[idx + 1], seeds[idx//2 + 1]) for idx in range(4)]
    fish = fisherFromVecs([fid_arr, *der_arr], np.array([0.02, 0.03]))
    show_fm_and_bias(fish)
    return fish

def plotLearningGraphs(history):
    ncols = 4
    fig, axes = plt.subplots(nrows=1, ncols= ncols, figsize=(3 * ncols + 2 , 3))
    lis = ['loss', 'lnfi', 'bias0', 'bias1']
    for idx in range(4) :
        axes[idx].plot(history[lis[idx]][1:], label = "Training")
        axes[idx].plot(history['val_' + lis[idx]][1:], label = "Validation")
        if "bias" in lis[idx]: axes[idx].axhline(0.2, linestyle = 'dotted', c = 'black')
        axes[idx].set_title(lis[idx])
    fig.legend(labels=['Training', 'Validation']) 

def run_imnn(vecs):
    fid_arr, der_arr = vecs[0], vecs[1:]
    all_vecs = np.stack([fid_arr[:7500], *der_arr])
    delta_theta = np.array([0.02, 0.03])
    comp = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation= tf.keras.layers.LeakyReLU(alpha=0.01)),
        tf.keras.layers.Dense(2)])


    imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 400, reg = tf.constant([0., 0.]),\
                           callbacks = [tf.keras.callbacks.EarlyStopping(
                                    patience = 3, restore_best_weights = True)],\
                           optimizer = tf.keras.optimizers.Adam(learning_rate= 1e-3),
                           data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
    fish = imnn_layer.computeFisher(all_vecs, delta_theta)

    return [fish.lnDetF.numpy(), np.round(fish.fractional_bias.numpy().flatten(), 2), fish]

## Benchmark from Jacky

In [8]:
mat = [[637509.6239556,  175715.25139639], [175715.25139638, 175536.25526324]]
print ("log FI = ", np.log(np.linalg.det(mat)))
print ("FM = ", np.round(np.array(mat).flatten(), 2))
print("Constraints = ", np.sqrt(np.diag(np.linalg.inv(mat))))

log FI =  25.118087792615572
FM =  [637509.62 175715.25 175715.25 175536.26]
Constraints =  [0.00147184 0.00280492]


# Histogram

## 15

In [9]:
directory_path = '/Users/karthikviswanathan/Downloads/imph/histogram_15_10'
print("Before compression")
fish = fisherAnalysis(directory_path)
print("\nAfter MOPED")
fish = fisherAnalysisMOPED(directory_path)

Before compression
log FI =  21.97
FM =  [157957.65  68116.18  68116.18  51378.42]
Fractional bias =  [0. 0.]
Constraints =  [0.0038447  0.00674129]

After MOPED
log FI =  21.96
FM =  [158717.68  68439.46  68439.46  51166.07]
Fractional bias =  [0. 0.]
Constraints =  [0.00385835 0.00679552]


### IMNN

In [ ]:
vecs = get_vecs(directory_path)
fid_arr, der_arr = vecs[0], vecs[1:]
all_vecs = np.stack([fid_arr[:7500], *der_arr])
delta_theta = np.array([0.02, 0.03])
comp = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(2)])
imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 200, reg = tf.constant([1., 1.5]),\
                       callbacks = [tf.keras.callbacks.EarlyStopping(
                                patience = 3, restore_best_weights = True)],\
                       data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
fish = imnn_layer.computeFisher(all_vecs, delta_theta)
show_fm_and_bias(fish)
plotLearningGraphs(comp.history.history)
 

## 100

In [11]:
directory_path = '/Users/karthikviswanathan/Downloads/imph/histogram_100_10'
print("Before compression")
fish = fisherAnalysis(directory_path)
print("\nAfter MOPED")
fish = fisherAnalysisMOPED(directory_path)

Before compression
log FI =  19.84
FM =  [83626.01 22501.68 22501.68 11011.78]
Fractional bias =  [0. 0.]
Constraints =  [0.00515398 0.01420315]

After MOPED
log FI =  19.83
FM =  [84107.71 22628.   22628.   10970.96]
Fractional bias =  [0. 0.]
Constraints =  [0.00516835 0.01431025]


### IMNN

In [ ]:
vecs = get_vecs(directory_path)
fid_arr, der_arr = vecs[0], vecs[1:]
all_vecs = np.stack([fid_arr[:7500], *der_arr])
delta_theta = np.array([0.02, 0.03])
comp = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation=leaky_relu),
    tf.keras.layers.Dense(2)])
imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 200, reg = tf.constant([1., 1.5]),\
                       callbacks = [tf.keras.callbacks.EarlyStopping(
                                patience = 3, restore_best_weights = True)],\
                       data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
fish = imnn_layer.computeFisher(all_vecs, delta_theta)
show_fm_and_bias(fish)
plotLearningGraphs(comp.history.history)

In [ ]:
loss_list = []
for _ in tqdm(range(10)):
    try : loss_list.append(run_imnn(vecs))
    except Exception as e: print("An error occured: ", e)

In [ ]:
print([item[0] for item in loss_list])

## 15 + 100

### Check if we are concatenating data vectors of the same simulation

In [ ]:
def get_files(directory_path):
    pickle_files = glob.glob(os.path.join(directory_path, '*.pkl'))
    file_names = [os.path.basename(file) for file in pickle_files]
    return file_names

directory_paths = ['/Users/karthikviswanathan/Downloads/imph/histogram_15_10', \
                   '/Users/karthikviswanathan/Downloads/imph/histogram_100_10']
pickle_files = [get_files(item) for item in directory_paths]
for a,b in zip(pickle_files[0], pickle_files[1]): 
    if a != b : print(a, b)


### Concatenating data vectors

In [14]:
v1, v2 = get_vecs('/Users/karthikviswanathan/Downloads/imph/histogram_15_10'), \
            get_vecs('/Users/karthikviswanathan/Downloads/imph/histogram_100_10')

In [15]:
seed = np.random.randint(1000)
vecs = [np.concatenate([shuffle_along_axis(va, seed = seed), \
                        shuffle_along_axis(vb, seed = seed)], axis = -1) for va, vb in zip(v1, v2)]

fish = fisherFromVecs(vecs, np.array([0.02, 0.03]))
show_fm_and_bias(fish)

print("\nAfter compressing")
fish = fisherMOPED(vecs, np.array([0.02, 0.03]))
show_fm_and_bias(fish)

log FI =  22.06
FM =  [165175.67  70890.33  70890.33  53529.9 ]
Fractional bias =  [0. 0.]
Constraints =  [0.00374517 0.00657879]

After compressing
log FI =  21.97
FM =  [159311.6   68380.83  68380.83  51249.83]
Fractional bias =  [0. 0.]
Constraints =  [0.00383275 0.00675754]


### IMNN

In [ ]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.01)
fid_arr, der_arr = vecs[0], vecs[1:]
all_vecs = np.stack([fid_arr[:7500], *der_arr])
delta_theta = np.array([0.02, 0.03])
comp = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation = leaky_relu),
    tf.keras.layers.Dense(2)])
imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 100, reg = tf.constant([0., 0.]),\
                       callbacks = [tf.keras.callbacks.EarlyStopping(
                                patience = 3, restore_best_weights = True)],
                       data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
fish = imnn_layer.computeFisher(all_vecs, delta_theta)
show_fm_and_bias(fish)
plotLearningGraphs(comp.history.history)

In [ ]:
for item in fish.summaries :
    print(np.cov(np.array(item).T).flatten())

### Additional checks

In [ ]:
loss_list = []
for _ in tqdm(range(10)):
    try : loss_list.append(run_imnn(vecs))
    except Exception as e: print("An error occured: ", e)
print([item[0] for item in loss_list if max(item[1]) < 0.2])

In [ ]:
summaries = loss_list[1][-1].summaries
for item in summaries :
    print(np.cov(np.array(item).T).flatten())

In [ ]:
summaries = loss_list[-2][-1].summaries
for item in summaries :
    print(np.cov(np.array(item).T).flatten())

# TopK

## 15

In [12]:
directory_path = '/Users/karthikviswanathan/Downloads/imph/topk_15_100'
print("Before compression")
fish = fisherAnalysis(directory_path)
print("\nAfter MOPED")
fish = fisherAnalysisMOPED(directory_path)

Before compression
log FI =  22.0
FM =  [150252.16  56263.13  56263.13  44979.73]
Fractional bias =  [0.01 0.01]
Constraints =  [0.0035383 0.0064669]

After MOPED
log FI =  21.68
FM =  [130155.3   48560.5   48560.5   38105.43]
Fractional bias =  [0. 0.]
Constraints =  [0.0038272  0.00707325]


### IMNN

In [ ]:
vecs = get_vecs(directory_path)
fid_arr, der_arr = vecs[0], vecs[1:]
all_vecs = np.stack([fid_arr[:7500], *der_arr])
delta_theta = np.array([0.02, 0.03])
comp = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(2)])
imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 200, reg = tf.constant([1., 1.5]),\
                       callbacks = [tf.keras.callbacks.EarlyStopping(
                                patience = 3, restore_best_weights = True)],\
                       data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
fish = imnn_layer.computeFisher(all_vecs, delta_theta)
show_fm_and_bias(fish)
plotLearningGraphs(comp.history.history)
 

## 100

In [13]:
directory_path = '/Users/karthikviswanathan/Downloads/imph/histogram_100_10'
print("Before compression")
fish = fisherAnalysis(directory_path)
print("\nAfter MOPED")
fish = fisherAnalysisMOPED(directory_path)

Before compression
log FI =  19.84
FM =  [83626.01 22501.68 22501.68 11011.78]
Fractional bias =  [0. 0.]
Constraints =  [0.00515398 0.01420315]

After MOPED
log FI =  19.79
FM =  [81053.73 22084.   22084.   10860.74]
Fractional bias =  [0. 0.]
Constraints =  [0.00525962 0.01436849]


### IMNN

In [ ]:
vecs = get_vecs(directory_path)
fid_arr, der_arr = vecs[0], vecs[1:]
all_vecs = np.stack([fid_arr[:7500], *der_arr])
delta_theta = np.array([0.02, 0.03])
comp = tf.keras.Sequential([
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(2)])
imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 200, reg = tf.constant([1., 1.5]),\
                       callbacks = [tf.keras.callbacks.EarlyStopping(
                                patience = 3, restore_best_weights = True)],\
                       data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
fish = imnn_layer.computeFisher(all_vecs, delta_theta)
show_fm_and_bias(fish)
plotLearningGraphs(comp.history.history)

## 15 + 100

### Check if we are concatenating data vectors of the same simulation

In [ ]:
def get_files(directory_path):
    pickle_files = glob.glob(os.path.join(directory_path, '*.pkl'))
    file_names = [os.path.basename(file) for file in pickle_files]
    return file_names

directory_paths = ['/Users/karthikviswanathan/Downloads/imph/topk_15_200', \
                   '/Users/karthikviswanathan/Downloads/imph/topk_100_200']
pickle_files = [get_files(item) for item in directory_paths]
for a,b in zip(pickle_files[0], pickle_files[1]): 
    if a != b : print(a, b)


In [16]:
v1, v2 = get_vecs('/Users/karthikviswanathan/Downloads/imph/topk_15_200'), \
            get_vecs('/Users/karthikviswanathan/Downloads/imph/topk_100_200')

In [18]:
seed = np.random.randint(1000)
vecs = [np.concatenate([shuffle_along_axis(va, seed = seed), \
                        shuffle_along_axis(vb, seed = seed)], axis = -1) for va, vb in zip(v1, v2)]

fish = fisherFromVecs(vecs, np.array([0.02, 0.03]))
show_fm_and_bias(fish)

print("\nAfter compressing")
fish = fisherMOPED(vecs, np.array([0.02, 0.03]))
show_fm_and_bias(fish)

log FI =  22.35
FM =  [179985.    64893.47  64893.47  51679.99]
Fractional bias =  [0.03 0.03]
Constraints =  [0.00318627 0.0059462 ]

After compressing
log FI =  20.93
FM =  [92787.61 34336.58 34336.58 25897.04]
Fractional bias =  [0. 0.]
Constraints =  [0.00459989 0.00870699]


### IMNN

In [ ]:
fid_arr, der_arr = vecs[0], vecs[1:]
all_vecs = np.stack([fid_arr[:7500], *der_arr])
delta_theta = np.array([0.02, 0.03])
comp = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation= 'relu'),
    tf.keras.layers.Dense(2)])


imnn_layer = IMNNLayer(comp, verbose = 0, epochs = 400, reg = tf.constant([0., 0.]),\
                       callbacks = [tf.keras.callbacks.EarlyStopping(
                                patience = 3, restore_best_weights = True)],\
                       optimizer = tf.keras.optimizers.Adam(learning_rate= 1e-3),
                       data_splits = [0.4, 0.2, 0.4], show_bias = True, show_fi = True)
fish = imnn_layer.computeFisher(all_vecs, delta_theta)


show_fm_and_bias(fish)
plotLearningGraphs(comp.history.history)

In [ ]:
loss_list = []
for _ in tqdm(range(10)):
    try : loss_list.append(run_imnn(vecs))
    except Exception as e: print("An error occured: ", e)
print([item[0] for item in loss_list if max(item[1]) < 0.2])